In [1]:
import os, glob, sys, re
import numpy as np
from prettytable import PrettyTable

In [2]:
f = open('ParticleData.html', 'r')
content = f.readlines()

In [3]:
# def isRelevantDecay(products):
#     DstSources = [411, 413, 10433, 10413, 10423, 415, 425, 20413, 20423]
#     idxDst = [ i for i,x in enumerate(np.abs(products)) if x in DstSources ]
#     if len(idxDst) == 0:
#         return False

    
#     idxHc = []
#     for i, x in enumerate(np.abs(products)):
#         if i == idxDst[0]:
#             continue
#         if (x % 1000)/100 == 4:
#             idxHc.append(i)
#     if len(idxHc) == 0:
#         return False
    
#     return True

def isRelevantDecay(products):
    if not 413 in products:
        return False
    
    return True

In [16]:
particlesData = {}
interestigDecays = []

pHeader = '[ ]*<p/>particle:[ ]*id="[0-9]+" name='
decayHeader = '[ ]*<br/>[ ]+channel: '
skipNext = 0
pId = None
for iln, ln in enumerate(content):
    if skipNext > 0:
        skipNext -= 1
        continue
    
    elif not ln.strip():
        continue
    
    elif pId is None and not re.match(pHeader, ln): 
        continue
        
    elif re.match(pHeader, ln):
        pData = ln[14:-1]
        addLines = 0
        while re.match(r'[ ]+[a-zA-Z0-9]+=".*"', content[iln+addLines+1]):
            addLines += 1
            pData += ' ' + content[iln+addLines].strip()
            
        skipNext = addLines
        for d in re.findall(r'[a-zA-Z]+="[/a-zA-z 0-9\.\*\-\+_\(\)]+"', pData):
            if not d: 
                continue
            if re.match('id="[0-9]+"', d):
                pId = int(d[4:-1])
                particlesData[pId] = {}
            else:
                k, v = re.split('=', d, maxsplit=1)
                if k.endswith('Type'):
                    k = k[:-4]
                v = v[1:-1]
                if k in ['charge', 'spin', 'col']:
                    v = int(v)
                elif k in ['m0']:
                    v = float(v)
                particlesData[pId][k] = v
    elif pId is not None and re.match(decayHeader, ln):
        data = ln[len(decayHeader):].strip()
        br = None
        products = []
        for aux in re.findall(r'[a-zA-Z]+="[ 0-9\.\-]+"', data):
            k, v = aux.split('=')
            v = v[1:-1]
            if k == 'bRatio': 
                br = float(v)
            elif k == 'products':
                products = [int(x) for x in v.split(' ') if x]
                
        flag = br > 0.01 and pId == 511
        for x in [413]:
            flag = flag and (x in np.abs(products))
        if flag:
            print 'Found:', pId, br, products
#         if isRelevantDecay(products) and br > 1e-2:
        if 413 in np.abs(products) and br > 1e-2 and (pId % 1000)/100 == 5:
            interestigDecays.append([pId, br, products])
#             print 'Found:', pId, br, products
#             break
    elif ln.strip() in ['</body>', '</html>', '<!-- Copyright (C) 2022 Torbjorn Sjostrand -->']:
        continue
    else:
        print pId
        print 'Line not recognized at', iln
        print '>',ln,'<'
        break

Found: 511 0.057 [12, -11, -413]
Found: 511 0.057 [14, -13, -413]
Found: 511 0.016 [16, -15, -413]
Found: 511 0.0126 [431, -413]
Found: 511 0.024 [433, -413]
Found: 511 0.012 [20213, -413]


In [17]:
print 'Found {} D*Hc decays'.format(len(interestigDecays))

table = PrettyTable()
table.field_names = ['Mother', 'Products', 'Br [1e-3]']

for motherId, br, products in interestigDecays:
    prodNames = ''
    for pid in products:
        if pid > 0:
            prodNames += particlesData[pid]['name'] + ' '
        else:
            prodNames += particlesData[-pid]['antiName'] + ' '
    out = [particlesData[motherId]['name'], prodNames[:-1], '{:.2f}'.format(1e3*br)]
    table.add_row(out)
print table

Found 8 D*Hc decays
+--------+-----------------+-----------+
| Mother |     Products    | Br [1e-3] |
+--------+-----------------+-----------+
|   B0   |   nu_e e+ D*-   |   57.00   |
|   B0   |  nu_mu mu+ D*-  |   57.00   |
|   B0   | nu_tau tau+ D*- |   16.00   |
|   B0   |     D_s+ D*-    |   12.60   |
|   B0   |    D*_s+ D*-    |   24.00   |
|   B0   |  a_1(1260)+ D*- |   12.00   |
|   B+   | D*- pi+ pi+ pi0 |   15.00   |
|  B_s0  |   D*_s- D*+ K0  |   15.00   |
+--------+-----------------+-----------+
